# Assignment 2: Language Modelling in Hangman

Student Name: Yingpeng Tan

Student ID: 739 674

## General info

<b>Due date</b>: Tuesday, 6 April 2021 5pm

<b>Submission method</b>: Canvas submission

<b>Submission materials</b>: completed copy of this iPython notebook

<b>Late submissions</b>: -10% per day (both week and weekend days counted)

<b>Marks</b>: 8% of mark for class (with 7% on correctness + 1% on quality and efficiency of your code)

<b>Materials</b>: See [Using Jupyter Notebook and Python page](https://canvas.lms.unimelb.edu.au/courses/121115/pages/using-jupyter-notebook-and-python?module_item_id=2681264) on Canvas (under Modules>Resources) for information on the basic setup required for this class, including an iPython notebook viewer and the python packages NLTK, Numpy, Scipy, Matplotlib, Scikit-Learn, and Gensim. We recommend installing all the data for NLTK, since you will need various parts of it to complete this assignment. You can also use any Python built-in packages, but do not use any other 3rd party packages (the packages listed above are all fine to use); if your iPython notebook doesn't run on the marker's machine, you will lose marks. <b> You should use Python 3</b>.  

To familiarize yourself with NLTK, here is a free online book:  Steven Bird, Ewan Klein, and Edward Loper (2009). <a href=http://nltk.org/book>Natural Language Processing with Python</a>. O'Reilly Media Inc. You may also consult the <a href=https://www.nltk.org/api/nltk.html>NLTK API</a>.

<b>Evaluation</b>: Your iPython notebook should run end-to-end without any errors in a reasonable amount of time, and you must follow all instructions provided below, including specific implementation requirements and instructions for what needs to be printed (please avoid printing output we don't ask for). You should edit the sections below where requested, but leave the rest of the code as is. You should leave the output from running your code in the iPython notebook you submit, to assist with marking. The amount each question is worth is explicitly given. 

You will be marked not only on the correctness of your methods, but also the quality and efficency of your code: in particular, you should be careful to use Python built-in functions and operators when appropriate and pick descriptive variable names that adhere to <a href="https://www.python.org/dev/peps/pep-0008/">Python style requirements</a>. If you think it might be unclear what you are doing, you should comment your code to help the marker make sense of it.

<b>Updates</b>: Any major changes to the assignment will be announced via Canvas. Minor changes and clarifications will be announced on the discussion board; we recommend you check it regularly.

<b>Academic misconduct</b>: For most people, collaboration will form a natural part of the undertaking of this homework, and we encourge you to discuss it in general terms with other students. However, this ultimately is still an individual task, and so reuse of code or other instances of clear influence will be considered cheating. We will be checking submissions for originality and will invoke the University’s <a href="http://academichonesty.unimelb.edu.au/policy.html">Academic Misconduct policy</a> where inappropriate levels of collusion or plagiarism are deemed to have taken place.

# Overview

In this homework, you'll be creating an 'artificial intelligence' player for the classic Hangman word guessing game. You will need to implement several different automatic strategies based on character-level n-gram language models. Your objective is to create an automatic player which makes the fewest mistakes.

# The Hangman Game (7 marks)

**Instructions**: The <a href="https://en.wikipedia.org/wiki/Hangman_(game)">Hangman game</a> is a simple game whereby one person thinks of a word, which they keep secret from their opponent, who tries to guess the word one character at a time. The game ends when the opponent makes more than a fixed number of incorrect guesses, or they figure out the secret word before then (in which case they *win*). 

Here's a simple version of the game. **No implementation is needed.**

In [1]:
def hangman(secret_word, guesser, max_mistakes=8, verbose=True, **guesser_args):
    """
        This function plays the hangman game with the provided guesser and returns the number of incorrect guesses. 
        
        secret_word: a string of lower-case alphabetic characters, i.e., the answer to the game
        guesser: a function which guesses the next character at each stage in the game
            The function takes a:
                mask: what is known of the word, as a string with _ denoting an unknown character
                guessed: the set of characters which already been guessed in the game
                guesser_args: additional (optional) keyword arguments, i.e., name=value
        max_mistakes: limit on length of game, in terms of number of allowed mistakes
        verbose: silent or verbose diagnostic prints
        guesser_args: keyword arguments to pass directly to the guesser function
    """
    secret_word = secret_word.lower()
    mask = ['_'] * len(secret_word)
    guessed = set()
    if verbose:
        print("Starting hangman game. Target is", ' '.join(mask), 'length', len(secret_word))
    
    mistakes = 0
    while mistakes < max_mistakes:
        if verbose:
            print("You have", (max_mistakes-mistakes), "attempts remaining.")
        guess = guesser(mask, guessed, **guesser_args)

        if verbose:
            print('Guess is', guess)
        if guess in guessed:
            if verbose:
                print('Already guessed this before.')
            mistakes += 1
        else:
            guessed.add(guess)
            if guess in secret_word and len(guess) == 1:
                for i, c in enumerate(secret_word):
                    if c == guess:
                        mask[i] = c
                if verbose:
                    print('Good guess:', ' '.join(mask))
            else:
                if len(guess) != 1:
                    print('Please guess with only 1 character.')
                if verbose:
                    print('Sorry, try again.')
                mistakes += 1
                
        if '_' not in mask:
            if verbose:
                print('Congratulations, you won.')
            return mistakes
        
    if verbose:
        print('Out of guesses. The word was', secret_word)    
    return mistakes

Here is a human guesser allowing interactive play.

In [2]:
def human(mask, guessed, **kwargs):
    """
    This is a simple function for manual play.
    """
    print('\nEnter your guess:')
    return input().lower().strip()

If you want to play hangman interactively, please set `interactive` to `True`. When submitting your solution, set to `False` so we can automatically run the whole notebook using `Run All`.

In [5]:
interactive = False

<b>For your testing:</b>

You can play the game interactively using the following command:

In [6]:
if interactive:
    hangman('whatever', human, 8, True)

### Question 1 (1.0 mark)

**Instructions**: We will use the words in NLTK's Brown corpus for training an artificial intelligence guessing algorithm, and for evaluating the quality of the algorithm.

Your first task is to compute the number of **unique word types** occurring in the Brown corpus, using `nltk.corpus.brown` and the `words` method, and select only words that are **entirely comprised of alphabetic characters**. You should also **lowercase the words**. Finally, randomly shuffle (`numpy.random.shuffle`) this collection of word types, and split them into disjoint training and testing sets. The test set should contain 1000 word types, and the rest should be in the  training set. Note that we are intentionally making the hangman game hard, as the AI will need to cope with test words that it has not seen before, hence it will need to learn generalisable patterns of characters to make reasonable predictions.

**Task**: Collect all unique word types from the Brown corpus, and produce `training_set` and `test_set`, 2 lists that contain 2 disjointed sets of words. Both `training_set` and `test_set` should be a python `list` (as initialised in the code). `test_set` must contain exactly 1000 word types.

**Check**: Use the assertion statements in <b>"For your testing"</b> below for the expected output.

In [7]:
from nltk.corpus import brown
import numpy as np

np.random.seed(1)

# training_set stores the rest word types for training
training_set = []
# test_set stores 1000 word types for testing
test_set = []


###
# Your answer BEGINS HERE
###

# Collect all unique word types from the Brown corpus
allUniqueWordTypes = []
for eachWord in brown.words():
    eachWord = eachWord.lower()
    if eachWord.isalpha() and eachWord not in allUniqueWordTypes:
        allUniqueWordTypes.append(eachWord)

# test_set must contain exactly 1000 word types, then training_set need to contain the rest of them
np.random.shuffle(allUniqueWordTypes)
training_set.extend(allUniqueWordTypes[:-1000])
test_set.extend(allUniqueWordTypes[-1000:])

###
# Your answer ENDS HERE
###

print("Number of word types in test =", len(test_set))
print("Number of word types in train =", len(training_set))

Number of word types in test = 1000
Number of word types in train = 39234


<b>For your testing:</b>

In [8]:
assert(len(training_set) > 35000 and len(training_set) < 45000)
assert(len(test_set) == 1000)

**Play the game**:

Let's see how good you are at this game! Try to guess a random word from the test set. It is surprisingly difficult (and addictive)! Don't forget to set `interactive = True`.

In [9]:
#play hangman using random words from test set
if interactive:
    hangman(np.random.choice(test_set), human, 8, True)

### Question 2 (1.0 mark)

**Instructions**: To set a baseline, your first AI attempt will be a trivial **random method**. For this you should implement a guessing method, similar to the `human` method above, i.e., using the same input arguments and returning a character. Your method should randomly choose a character from the range `a ... z` after excluding the characters that have already been guessed in the current game (all subsequent AI approaches should also exclude previous guesses).

To help you measure the performance of this (and later) guesser, a `test_guesser` method that takes a guesser and measures the average number of incorrect guesses made over all the words in the `test_set` is provided to you. 

**Task**: Complete the `random_guesser` method. It should return a random character from the English alphabets.

**Check**: Use the assertion statements in <b>"For your testing"</b> below for the expected output.

In [10]:
def test_guesser(guesser, test=test_set):
    """
        This function takes a guesser and measures the average number of incorrect guesses made over all the words in the test_set. 
    """
    total = 0
    for word in test:
        total += hangman(word, guesser, 26, False)
    return total / float(len(test))

In [11]:
def random_guesser(mask, guessed, **kwargs):
    
    ###
    # Your answer BEGINS HERE
    ###
    return np.random.choice(list(set('abcdefghijklmnopqrstuvwxyz') - guessed))
    ###
    # Your answer ENDS HERE
    ###

random_word = np.random.choice(test_set)
print("Guessing word =", random_word)
print("Number of mistakes made by the random guesser =", hangman(random_word, random_guesser, 26, False))

result = test_guesser(random_guesser)
print("\nTesting the random guesser using every word in test set")
print("Average number of incorrect guesses: ", result)

Guessing word = status
Number of mistakes made by the random guesser = 18

Testing the random guesser using every word in test set
Average number of incorrect guesses:  16.681


<b>For your testing:</b>

In [12]:
assert(result > 10 and result < 20)

### Question 3 (1.0 mark)

**Instructions:** As your first real AI, you should train a **unigram language model** over the training set. This requires you to find the frequencies of characters over all training words. Using this model, you should write a guesser that returns the character with the highest probability. Remember to exclude already guessed characters. 

**Task**: Collect the frequencies of characters and store them in `unigram_counts` (use the first answer space). Complete the `unigram_guesser` method. Note that it takes `unigram_counts` as an additional argument (use the second answer space).

**Check**: Use the assertion statements in <b>"For your testing"</b> below for the expected output.

In [46]:
unigram_counts = None

###
# Your answer BEGINS HERE
###

# Collect the frequencies of characters and store them in unigram_counts
# Ref: Python Counter https://docs.python.org/3/library/collections.html
from collections import Counter
unigram_counts = Counter("".join(training_set))

# In rare sisutations, put the freq of the characters to 0 if it didn't appear in the training set
for eachCharacter in "abcdefghijklmnopqrstuvwxyz":
    if eachCharacter not in unigram_counts:
        unigram_counts[eachCharacter]=0

###
# Your answer ENDS HERE
###

def unigram_guesser(mask, guessed, unigram_counts=unigram_counts):

    ###
    # Your answer BEGINS HERE
    ###
    # Unigram: Find the the Character with the maximum freqency. If multiple characters share the same maximum freq, choose randomly.
    
    maxFreq = 0
    maxFreqChar = []
    
    for eachChar, eachCharFreq in unigram_counts.items():
        if eachChar in guessed:
            continue
        if eachCharFreq > maxFreq:
            maxFreqChar = [eachChar]
            maxFreq = eachCharFreq
        elif eachCharFreq == maxFreq:
            maxFreqChar.append(eachChar)
    return np.random.choice(maxFreqChar)

    ###
    # Your answer ENDS HERE
    ###


result = test_guesser(unigram_guesser)
print("Testing the unigram guesser using every word in test set")
print("Average number of incorrect guesses: ", result)

Testing the unigram guesser using every word in test set
Average number of incorrect guesses:  10.808


<b>For your testing:</b>

In [14]:
assert(result > 5 and result < 15)

### Question 4 (1.0 mark)

**Instructions:** The length of the secret word is an important clue that we might exploit. Different lengths tend to have different distributions over characters, e.g., short words are less likely to have suffixes or prefixes. You should incorporate this idea by conditioning the unigram model on the length of the secret word, i.e.,  having a **different unigram model for each length**. You will need to be a little careful at test time, to be robust to the situation that you encounter a word length that you didn't see in training. In such a case, your method should behave like the previous `unigram_guesser` in Question 3 (i.e., it guesses characters based on unigram frequencies, unconditioned by the word length).

**Task**: Collect the frequencies of characters conditioned on the word length and store them in `unigram_counts_by_length` (use the first answer space). Complete the `unigram_length_guesser` method (use the second answer space).

**Check**: Use the assertion statements in <b>"For your testing"</b> below for the expected output.

In [47]:
unigram_counts_by_length = None

###
# Your answer BEGINS HERE
###

# Collect the frequencies of characters conditioned on the word length and store them in unigram_counts_by_length

from collections import Counter
unigram_counts_by_length = dict()

unigramModelDict = dict()
for eachWord in training_set:
    eachWordLength = len(eachWord)
    unigramModelDict [eachWordLength] = unigramModelDict.get(eachWordLength, "") + eachWord

for eachWordLength, eachWord in unigramModelDict.items():
    charCountInEachWord = Counter(eachWord)
    for eachChar in "abcdefghijklmnopqrstuvwxyz":
        if eachChar not in charCountInEachWord:
            charCountInEachWord[eachChar]=0
    unigram_counts_by_length[eachWordLength]=charCountInEachWord
    

###
# Your answer ENDS HERE
###

def unigram_length_guesser(mask, guessed, unigram_counts_by_length=unigram_counts_by_length, unigram_counts=unigram_counts):
    ###
    # Your answer BEGINS HERE
    ###
    wordLength = len(mask)
    if wordLength in unigram_counts_by_length:
        return unigram_guesser(mask,guessed, unigram_counts=unigram_counts_by_length[wordLength])
    
    # In the situation that encountering a word length that you didn't see in training. use previous unigram_guesser in Question 3
    else:
        return unigram_guesser(mask, guessed, unigram_counts=unigram_counts)
    ###
    # Your answer ENDS HERE
    ###

result = test_guesser(unigram_length_guesser)
print("Testing the length-conditioned unigram guesser using every word in test set")
print("Average number of incorrect guesses: ", result)

Testing the length-conditioned unigram guesser using every word in test set
Average number of incorrect guesses:  10.708


<b>For your testing:</b>

In [16]:
assert(result > 5 and result < 15)

### Question 5 (1.0 mark)

**Instructions:** Now for the next challenge, you'll build a **bigram language model** over characters. The order of characters is obviously important, yet this wasn't incorporated in any of the above models. Knowing that the word has the sequence `n _ s s` is a pretty strong clue that the missing character might be `e`. Similarly the distribution over characters that start or end a word are highly biased (e.g., toward common prefixes and suffixes, like *un-*, *-ed* and *-ly*).

The task here is to develop a bigram language model over characters, and train it over the training words. Remember to be careful when handling the start of each word properly, e.g., by padding with a special starting symbol such as `$`. Do we also need a special ending symbol? That's for you to decide.

Your bigram guesser should apply your language model to each blank position in the secret word by using its left context character. For example, in the partial word `e _ c _ b _ _` we know the left context for the first three blanks, but have no known left context for the last blank. In the case for the last blank, you should revert to using a unigram language model (since there's no context for us to use the bigram model). You should sum up the probability distribution (over all alphabets from <i>a</i> to <i>z</i>) for the 4 blanks, and select the alphabet with the highest probability that hasn't been guessed.

**Task**: Collect frequency counts that are necessary for building a bigram language model and store them in bigram_counts; feel free to add new objects if needed (use the first answer space). Complete the `bigram_guesser` method (use the second answer space). Note that the method currently only has one additional argument (`bigram_counts`), but you are free to add additional arguments.

In [61]:
bigram_counts = None

###
# Your answer BEGINS HERE
###

# Collect frequency counts that are necessary for building a bigram language model and store them in bigram_counts

from collections import Counter
from collections import defaultdict
bigram_counts = defaultdict(Counter)


for eachWord in training_set:
    #  Padding with a special starting symbol such as $
    eachWordList = ["$"] +list(eachWord)
    # The zip() function returns a zip object, which is an iterator of tuples where the first item in each passed
    # iterator is paired together, and then the second item in each passed iterator are paired together etc.
    # eachWordList[:-1] -- first character from index 0 to n-2
    # eachWordList[1:] -- second character from index 1 to n-1
    bigramList = zip(eachWordList[:-1],eachWordList[1:])
    for eachBigram in bigramList:
        firstCharacter, secondCharacter = eachBigram
        bigram_counts[firstCharacter][secondCharacter] = bigram_counts[firstCharacter][secondCharacter] +1

# Calculate the Probabilities
# Different from counts/frequencies, Probabilities can be added and compared
sumOfUnigramCounts = float(sum(unigram_counts.values()))
    
# Prepare the Probabilities of Unigram
unigramProbability = dict()
for eachCharacter, eachCharacterFreq in unigram_counts.items():
    unigramProbability[eachCharacter] = eachCharacterFreq / sumOfUnigramCounts
    
###
# Your answer ENDS HERE
###
    

def bigram_guesser(mask, guessed, bigram_counts=bigram_counts, unigramProbability=unigramProbability): # add extra arguments if needed
    ###
    # Your answer BEGINS HERE
    ###
    allCharacterProbability = dict()
    useUnigram = False
    processedFirstCharacterList = []
    paddedMask = ["$"] + mask
    
    # Initialize allCharacterProbability with all zero
    for eachCharacter in "abcdefghijklmnopqrstuvwxyz":
        allCharacterProbability[eachCharacter] = 0
            
    # parsing from the second character to the end of word
    for eachIndex in range(1, len(mask)+1):
        if paddedMask[eachIndex] != "_":
            continue
        firstCharacter = paddedMask[eachIndex - 1]
        
        # If the word is like "[c1] _", use bigram
        if firstCharacter != "_" and firstCharacter not in processedFirstCharacterList:
            possibleCharatersCount = float(sum(bigram_counts[firstCharacter].values()))
            processedFirstCharacterList.append(firstCharacter)
            for eachCharacter, eachCharacterFreq in bigram_counts[firstCharacter].items():
                # find the sum of the probabilities on each space
                allCharacterProbability[eachCharacter] = allCharacterProbability.get(eachCharacter, 0) + eachCharacterFreq / possibleCharatersCount
        
        # If the "_" is after a "_", then use unigram
        elif firstCharacter == "_":
            useUnigram = True
                
    # The probability of the Unigram will only be added once for better performance
    if useUnigram:
        for eachCharacter, eachCharacterProb in unigramProbability.items():
            allCharacterProbability[eachCharacter] = allCharacterProbability.get(eachCharacter, 0) + eachCharacterProb
    
    return  unigram_guesser(mask, guessed, unigram_counts=allCharacterProbability)
    ###
    # Your answer ENDS HERE
    ###

result = test_guesser(bigram_guesser)
print("Testing the bigram guesser using every word in test set")
print("Average number of incorrect guesses: ", result)

Testing the bigram guesser using every word in test set
Average number of incorrect guesses:  9.096


### Question 6 (1.5 mark)

**Instructions:** You should try to develop a more effective AI for hangman. Feel free to engage your creativity here! Possibilities include better conditioning on the length of the word, fancier smoothing methods, ngram models and bidirectional models (lecture 8). Have fun!

You will be marked based on the performance of your AI model, using a pre-made training and test set (created using a secret seed). Let x be the average number of mistakes in the test set, you will score:
* 1.5 mark if x < 8.0
* 1.0 mark if 8.0 <= x < 8.5
* 0.5 mark if 8.5 <= x < 8.8
* 0.0 mark if x >= 8.8

Note: when testing your AI model's performance, you may want to consider trying different training/test splits (using different seeds) to have a better understanding of its performance, as there will be some variance to its performance depending on the training/test split.

**Task** Complete the `my_amazing_ai_guesser` method, which implements a better language model for hangman.

In [62]:
###
# Your answer BEGINS HERE
###

# Bidirectional Bigram
# Calculate the probability based on the last character (c_x-1, the first character) and the next chracter (c_x+1 the third character)
# P(c_x|c_x-1 c_x+1)

from collections import Counter
from collections import defaultdict
bidirectionalBigramCounts = defaultdict(lambda: defaultdict(Counter))

for eachWord in training_set:
    #  Padding with a special starting symbol $ and a special ending symbol #
    eachWordList = ["$"] + list(eachWord) + ["#"]
    # The zip() function returns a zip object, which is an iterator of tuples where the first item in each passed
    # iterator is paired together, and then the second item in each passed iterator are paired together etc.
    # eachWordList[:-2] -- first character from index 0 to n-3
    # eachWordList[1:-1] -- second character from index 1 to n-2
    # eachWordList[2:] -- third character from index 3 to n-1
    bidirectionalBigramList = zip(eachWordList[:-2], eachWordList[1:-1], eachWordList[2:])
    for eachBidirectionalBigram in bidirectionalBigramList:
        firstCharacter, secondCharacter, thirdCharacter = eachBidirectionalBigram
        bidirectionalBigramCounts[firstCharacter][thirdCharacter][secondCharacter] = bidirectionalBigramCounts[firstCharacter][thirdCharacter][secondCharacter] + 1

def my_amazing_ai_guesser(mask, guessed, bidirectionalBigramCounts=bidirectionalBigramCounts, bigram_counts=bigram_counts, unigramProbability=unigramProbability):
    allCharacterProbability = dict()
    useUnigram = False
    processedFirstCharacters = []
    processedFirstThirdCharacters = []
    paddedMask = ["$"] + mask + ["#"]
    
    for eachCharacter in "abcdefghijklmnopqrstuvwxyz":
        allCharacterProbability[eachCharacter] = 0
    
    # parsing from the second character to the end of word
    for eachIndex in range(1, len(mask) + 1):
        if paddedMask[eachIndex] != "_":
            continue
        
        firstCharacter = paddedMask[eachIndex - 1]
        thirdCharacter = paddedMask[eachIndex + 1]
        firstThirdCharacterPair = firstCharacter + "_" + thirdCharacter
        
        # If the word is like "[c1]_ _", use bigram
        if firstCharacter != "_" and thirdCharacter == "_" and firstCharacter not in processedFirstCharacters:
            possibleFirstCharacterCounts = float(sum(bigram_counts[firstCharacter].values()))
            processedFirstCharacters.append(firstCharacter)
            for eachCharacter, eachCharacterFreq in bigram_counts[firstCharacter].items():
                allCharacterProbability[eachCharacter] = allCharacterProbability.get(eachCharacter, 0) + eachCharacterFreq / possibleFirstCharacterCounts
        
        # If the word is like "[c1] _ [c3]", use bidirectional bigram
        elif firstCharacter != "_" and thirdCharacter != "_" and firstThirdCharacterPair not in processedFirstThirdCharacters:
            CharacterCountsFirstThirdKnown = float(sum(bidirectionalBigramCounts[firstCharacter][thirdCharacter].values()))
            processedFirstThirdCharacters.append(firstThirdCharacterPair)
            for eachCharacter, eachCharacterFreq in bidirectionalBigramCounts[firstCharacter][thirdCharacter].items():
                allCharacterProbability[eachCharacter] = allCharacterProbability.get(eachCharacter, 0) + eachCharacterFreq / CharacterCountsFirstThirdKnown
        
        # If the word is like "_ _ [c3]" or "_ _ _", use unigram
        elif firstCharacter == "_":
            useUnigram = True

    if useUnigram:
        for eachCharacter, eachCharacterProb in unigramProbability.items():
            allCharacterProbability[eachCharacter] = allCharacterProbability.get(eachCharacter, 0) + eachCharacterProb
            
            
    return  unigram_guesser(mask, guessed, unigram_counts=allCharacterProbability)

###
# Your answer ENDS HERE
###

result = test_guesser(my_amazing_ai_guesser)
print("Testing my amazing AI guesser using every word in test set")
print("Average number of incorrect guesses: ", result)


Testing my amazing AI guesser using every word in test set
Average number of incorrect guesses:  7.739


### Question 7 (0.5 mark)

**Instructions:** Explain your approach and discuss your result below. Please keep your explanation to a short paragraph.

##### Your answer BEGINS HERE

The algorithm I designed takes the advantage of bigram, bidirectional bigram and unigram algorithms. It tries to use as much information as possible to make the most evidently reasonable prediction. If the characters before and after the “_" are known ("[c1] _ [c3]"), the algorithm will try to predict the guess using P(c_x|c_x-1 c_x+1), the bidirectional bigram; If only the character before the "_" is known ("[c1] _ _"), the algorithm will back off and predict by P(c_x|c_x-1) only, the bigram algorithm; If the character before "_" is not known ("_ _"), the algorithm will back off to P(c_x), the unigram algorithm. All those three algorithms bring the incorrect guesses to as low as 7.739 (less than 8). Throughout this assignment, the random guess algoritm has the worst performance at approx. 16 incorrect rate, and unigram is much better at 11 incorrect. While bigram with unigram back off brings the incorrect rate to even lower at 10. After adding up bidirectional bigram, the incorrect rate goes to the lowest ever at 7.7. Generally speaking, the more relevant information is ultilised correctly, the better performance of the algorithm will be.


##### Your answer ENDS HERE